In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST


In [6]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_set = MNIST(root='./data', train=True, download=True, transform=transform)
test_set = MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)


Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [8]:
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [9]:
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

[1,   100] loss: 0.628
[1,   200] loss: 0.164
[1,   300] loss: 0.123
[1,   400] loss: 0.099
[1,   500] loss: 0.079
[1,   600] loss: 0.071
[1,   700] loss: 0.068
[1,   800] loss: 0.058
[1,   900] loss: 0.066
[2,   100] loss: 0.049
[2,   200] loss: 0.039
[2,   300] loss: 0.046
[2,   400] loss: 0.042
[2,   500] loss: 0.040
[2,   600] loss: 0.047
[2,   700] loss: 0.040
[2,   800] loss: 0.043
[2,   900] loss: 0.043
[3,   100] loss: 0.021
[3,   200] loss: 0.033
[3,   300] loss: 0.030
[3,   400] loss: 0.035
[3,   500] loss: 0.030
[3,   600] loss: 0.032
[3,   700] loss: 0.029
[3,   800] loss: 0.032
[3,   900] loss: 0.026
[4,   100] loss: 0.016
[4,   200] loss: 0.021
[4,   300] loss: 0.018
[4,   400] loss: 0.024
[4,   500] loss: 0.025
[4,   600] loss: 0.019
[4,   700] loss: 0.037
[4,   800] loss: 0.024
[4,   900] loss: 0.027
[5,   100] loss: 0.013
[5,   200] loss: 0.018
[5,   300] loss: 0.013
[5,   400] loss: 0.014
[5,   500] loss: 0.014
[5,   600] loss: 0.017
[5,   700] loss: 0.015
[5,   800] 

In [10]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy on the test set: %d %%' % (100 * correct / total))


Accuracy on the test set: 98 %
